In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO

sns.set_theme(palette='colorblind', font_scale=1.1)
palette = sns.color_palette().as_hex()

base_folder = Path(os.path.expanduser('~')) / 'workspace' / 'chahrazad'

/var/folders/xd/714yy8tn5vz90_wtnn7vglhc0000gq/T/ipykernel_1985/2697004585.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Extract relevant candidates from proteome

In [2]:
proteins = SeqIO.to_dict(SeqIO.parse(base_folder / 'mass_spec_output' / 'merged_contigs_S5a1.fa', 'fasta'))
len(proteins)

3898

In [3]:
mass_spec_df = pd.read_csv(base_folder / 'mass_spec_output' / '2023_03_11_quant_table_for_ID_fr_2.csv')
mass_spec_df['protein_id'] = mass_spec_df['PG.FastaHeaders'].apply(lambda v: v[1:].split(' ')[0].strip())
mass_spec_df['description'] = mass_spec_df['PG.FastaHeaders'].apply(lambda v: v.split(' ')[1].strip())
mass_spec_df.head()

,Unnamed: 0,SignalP,PG.FastaHeaders,PG.MolecularWeight,A_-6,A_-5,A_-4,A_-3,A_-2,A_-1,...,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,protein_id,description
0,NaN,NaN,>gnl|extdb|pgaptmp_000061_3 twin-arginine tran...,18418.65,0.498328,0.344635,0.163307,0.064468,0.028929,0.009794,...,0.007038,0.005093,0.002355,0.003894,0.002137,0.000726,0.001508,0.000953,gnl|extdb|pgaptmp_000061_3,twin-arginine
1,NaN,NaN,>gnl|extdb|pgaptmp_000109 50S ribosomal protei...,10599.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.310365,0.120477,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,gnl|extdb|pgaptmp_000109,50S
2,NaN,NaN,>gnl|extdb|pgaptmp_000146 multiprotein-bridgin...,18936.81,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,gnl|extdb|pgaptmp_000146,multiprotein-bridging
3,NaN,NaN,>gnl|extdb|pgaptmp_000165 CDC48 family AAA ATP...,82483.44,0.000000,0.569850,1.000000,0.596697,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,gnl|extdb|pgaptmp_000165,CDC48
4,NaN,Sec/SPI,>gnl|extdb|pgaptmp_000187_2 hypothetical prote...,43409.58,0.761900,1.000000,0.779876,0.312693,0.233290,0.119024,...,0.038653,0.022015,0.008348,0.005200,0.004847,0.003765,0.003090,0.003340,gnl|extdb|pgaptmp_000187_2,hypothetical


## Concatenate all-vs-all fasta files

`make_input_fasta.py` produces one file per bait. In this case it makes sense to pull them all into one file.

In [73]:
output_file = base_folder / 'S5a-1_candidate_analysis' / 'all_vs_all_pulldown.fasta'

all_records = []
for f in (base_folder / 'S5a-1_candidate_analysis').iterdir():
    if f.is_file() and f.name.endswith('.fasta') and f.name.startswith('gnl|extdb|'):
        for record in SeqIO.parse(f, 'fasta'):
            all_records.append(record)

with output_file.open('w') as f_out:
    SeqIO.write(all_records, f_out, 'fasta')

## Process pulldown results

In [4]:
scores_path = base_folder / 'S5a-1_candidate_analysis' / 'all_vs_all_scores.csv'
scores_df = pd.read_csv(scores_path)
scores_df.head()

,id,plddt,ptm,iptm,dockq,confidence
0,gnl_extdb_pgaptmp_001289_1__gnl_extdb_pgaptmp_...,92.07,0.89,0.88,0.7207,0.882
1,gnl_extdb_pgaptmp_002495_1__gnl_extdb_pgaptmp_...,93.86,0.89,0.86,0.7150,0.866
2,gnl_extdb_pgaptmp_000386_2__gnl_extdb_pgaptmp_...,88.16,0.77,0.66,0.2008,0.682
3,gnl_extdb_pgaptmp_001700_1__gnl_extdb_pgaptmp_...,83.81,0.66,0.65,0.1533,0.652
4,gnl_extdb_pgaptmp_000165_1__gnl_extdb_pgaptmp_...,78.20,0.59,0.65,0.2941,0.638


In [13]:
scores_df['query'] = scores_df['id'].apply(lambda v: v.split('__')[0].replace('gnl_extdb_', 'gnl|extdb|'))
scores_df['target'] = scores_df['id'].apply(lambda v: v.split('__')[1].replace('gnl_extdb_', 'gnl|extdb|'))
scores_df['query_description'] = scores_df['query'].apply(lambda v: proteins[v].description.replace(' [Haloferax sp. s5a-1]', '').replace(v, '').strip())
scores_df['target_description'] = scores_df['target'].apply(lambda v: proteins[v].description.replace(' [Haloferax sp. s5a-1]', '').replace(v, '').strip())
scores_df.head()

,id,plddt,ptm,iptm,dockq,confidence,query,target,query_description,target_description
0,gnl_extdb_pgaptmp_001289_1__gnl_extdb_pgaptmp_...,92.07,0.89,0.88,0.7207,0.882,gnl|extdb|pgaptmp_001289_1,gnl|extdb|pgaptmp_001289_1,glycine--tRNA ligase,glycine--tRNA ligase
1,gnl_extdb_pgaptmp_002495_1__gnl_extdb_pgaptmp_...,93.86,0.89,0.86,0.7150,0.866,gnl|extdb|pgaptmp_002495_1,gnl|extdb|pgaptmp_002495_1,aldehyde dehydrogenase family protein,aldehyde dehydrogenase family protein
2,gnl_extdb_pgaptmp_000386_2__gnl_extdb_pgaptmp_...,88.16,0.77,0.66,0.2008,0.682,gnl|extdb|pgaptmp_000386_2,gnl|extdb|pgaptmp_000386_2,"1,2-phenylacetyl-CoA epoxidase subunit A (plas...","1,2-phenylacetyl-CoA epoxidase subunit A (plas..."
3,gnl_extdb_pgaptmp_001700_1__gnl_extdb_pgaptmp_...,83.81,0.66,0.65,0.1533,0.652,gnl|extdb|pgaptmp_001700_1,gnl|extdb|pgaptmp_000564_1,thioredoxin family protein,CBS domain-containing protein
4,gnl_extdb_pgaptmp_000165_1__gnl_extdb_pgaptmp_...,78.20,0.59,0.65,0.2941,0.638,gnl|extdb|pgaptmp_000165_1,gnl|extdb|pgaptmp_002090_1,CDC48 family AAA ATPase,alpha/beta hydrolase


In [14]:
scores_df.to_csv(base_folder / 'S5a-1_candidate_analysis' / 'all_vs_all_scores_annotated.csv', index=False)